In [1]:
import numpy as np

# Task 1
- Write a program to implement Needleman-Wunsch for proteins
- You will need the blosum50 scoring matrix
- You can use any programming language
- Run this on HEAGAWGHEE versus PAWHEAE
- Compare this to page 23 in lecture 5
- Match the protein sequence SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL with PSPTMEAVTSVEASTASHPHSTSSYFATTYYHLY

In [19]:
# read blosum 50 matrix
blosum50_matrix = np.loadtxt('./blosum50.txt')
def blosumtodict(blosum50_matrix):
    blosum50_dict = {}
    string_list = 'ARNDCQEGHILKMFPSTWYV'
    for i, ichar in enumerate(string_list):
        for j, jchar in enumerate(string_list):
            blosum50_dict[(ichar, jchar)] = blosum50_matrix[i, j]
    return blosum50_dict
blosum50 = blosumtodict(blosum50_matrix)
blosum50
    

{('A', 'A'): 5.0,
 ('A', 'R'): -2.0,
 ('A', 'N'): -1.0,
 ('A', 'D'): -2.0,
 ('A', 'C'): -1.0,
 ('A', 'Q'): -1.0,
 ('A', 'E'): -1.0,
 ('A', 'G'): 0.0,
 ('A', 'H'): -2.0,
 ('A', 'I'): -1.0,
 ('A', 'L'): -2.0,
 ('A', 'K'): -1.0,
 ('A', 'M'): -1.0,
 ('A', 'F'): -3.0,
 ('A', 'P'): -1.0,
 ('A', 'S'): 1.0,
 ('A', 'T'): 0.0,
 ('A', 'W'): -3.0,
 ('A', 'Y'): -2.0,
 ('A', 'V'): 0.0,
 ('R', 'A'): -2.0,
 ('R', 'R'): 7.0,
 ('R', 'N'): -1.0,
 ('R', 'D'): -2.0,
 ('R', 'C'): -4.0,
 ('R', 'Q'): 1.0,
 ('R', 'E'): 0.0,
 ('R', 'G'): -3.0,
 ('R', 'H'): 0.0,
 ('R', 'I'): -4.0,
 ('R', 'L'): -3.0,
 ('R', 'K'): 3.0,
 ('R', 'M'): -2.0,
 ('R', 'F'): -3.0,
 ('R', 'P'): -3.0,
 ('R', 'S'): -1.0,
 ('R', 'T'): -1.0,
 ('R', 'W'): -3.0,
 ('R', 'Y'): -1.0,
 ('R', 'V'): -3.0,
 ('N', 'A'): -1.0,
 ('N', 'R'): -1.0,
 ('N', 'N'): 7.0,
 ('N', 'D'): 2.0,
 ('N', 'C'): -2.0,
 ('N', 'Q'): 0.0,
 ('N', 'E'): 0.0,
 ('N', 'G'): 0.0,
 ('N', 'H'): 1.0,
 ('N', 'I'): -3.0,
 ('N', 'L'): -4.0,
 ('N', 'K'): 0.0,
 ('N', 'M'): -2.0,
 ('N', 'F'

In [53]:
def needleman_wunsch(string_a, string_b, d, s_dict):
    F = np.zeros((len(string_a)+1, len(string_b)+1))
    for i,j in np.ndindex(F.shape):
        if i == 0:
            F[0, j] = d * j
        elif j == 0:
            F[i, 0] = d * i
        elif (i > 0 and j > 0):
            match = F[i-1, j-1] + s_dict[(string_a[i-1], string_b[j-1])]
            delete = F[i-1, j] + d
            insert = F[i, j-1] + d
            F[i, j] = max(match, delete, insert)
    alignment_a = ''
    alignment_b = ''
    i = len(string_a)
    j = len(string_b)
    while (i > 0 or j > 0):
        if (i > 0 and j > 0 and F[i, j] == F[i-1,j-1] + s_dict[(string_a[i-1], string_b[j-1])]):
            alignment_a = string_a[i-1] + alignment_a
            alignment_b = string_b[j-1] + alignment_b
            i = i - 1
            j = j - 1
        elif (i > 0 and F[i, j] == F[i-1, j] + d):
            alignment_a = string_a[i-1] + alignment_a
            alignment_b = '-' + alignment_b
            i = i - 1
        else:
            alignment_a = '-' + alignment_a
            alignment_b = string_b[j-1] + alignment_b
            j = j - 1
            
    return alignment_a, alignment_b

In [54]:
needleman_wunsch('HEAGAWGHEE', 'PAWHEAE', -8, blosum50)
needleman_wunsch('SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL', 'PSPTMEAVTSVEASTASHPHSTSSYFATTYYHLY', -8, blosum50)

('-SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL-', 'PSPTMEAVTSVEA-STASHPHSTSSYFATTYYHLY')